# Model Monitoring with Evidently AI

This notebook demonstrates comprehensive model monitoring capabilities using Evidently AI for automated drift detection and performance analysis. The implementation showcases production-ready monitoring workflows for machine learning systems.

In [ ]:
import pandas as pd
import numpy as np
import sys
import os
from datetime import datetime

# add src directory to path
sys.path.append('../src')

from monitoring import generate_drift_report, load_model_and_data

## Data Loading and Preparation

Load baseline reference data and simulated drift scenarios for comprehensive monitoring analysis.

In [ ]:
# load baseline data
baseline_data = pd.read_csv('../data/processed/housing_baseline.csv')
print(f"baseline data shape: {baseline_data.shape}")

# load drift scenarios
drift_scenario_1 = pd.read_csv('../data/processed/housing_drift_scenario_1.csv')
drift_scenario_2 = pd.read_csv('../data/processed/housing_drift_scenario_2.csv')

print(f"drift scenario 1 shape: {drift_scenario_1.shape}")
print(f"drift scenario 2 shape: {drift_scenario_2.shape}")

## Evidently AI Drift Report Generation

Generate comprehensive drift detection reports using Evidently AI's automated monitoring capabilities. These reports provide detailed analysis of data distribution changes and feature drift patterns.

In [ ]:
# generate report for scenario 1 - feature drift
report_1 = generate_drift_report(
    reference_data=baseline_data,
    current_data=drift_scenario_1,
    report_name='drift_scenario_1'
)

print("drift report 1 generated successfully")
print(f"report saved to: ../reports/drift_scenario_1_{datetime.now().strftime('%Y%m%d_%H%M')}.html")

In [ ]:
# generate report for scenario 2 - distribution change
report_2 = generate_drift_report(
    reference_data=baseline_data,
    current_data=drift_scenario_2, 
    report_name='drift_scenario_2'
)

print("drift report 2 generated successfully")
print(f"report saved to: ../reports/drift_scenario_2_{datetime.now().strftime('%Y%m%d_%H%M')}.html")

## Statistical Drift Analysis

Perform detailed statistical analysis of drift detection results to identify specific features experiencing distribution changes and quantify the magnitude of detected drift.

In [ ]:
# check drift detection metrics
from evidently.metric_preset import DataDriftPreset
from evidently.report import Report

# create detailed drift analysis
drift_report = Report(metrics=[
    DataDriftPreset()
])

# run on scenario 1
drift_report.run(reference_data=baseline_data, current_data=drift_scenario_1)
drift_results = drift_report.as_dict()

print("data drift analysis - scenario 1:")
print(f"dataset drift detected: {drift_results['metrics'][0]['result']['dataset_drift']}")
print(f"number of drifted features: {drift_results['metrics'][0]['result']['number_of_drifted_columns']}")
print(f"share of drifted features: {drift_results['metrics'][0]['result']['share_of_drifted_columns']:.2%}")

In [ ]:
# analyze which specific features are drifting
drift_by_feature = drift_results['metrics'][0]['result']['drift_by_columns']

print("\nfeature-level drift analysis:")
for feature, drift_info in drift_by_feature.items():
    if drift_info['drift_detected']:
        print(f"{feature}: drift detected (p-value: {drift_info['stattest_threshold']:.4f})")
    else:
        print(f"{feature}: no drift detected")

## Model Performance Monitoring

Evaluate model prediction performance on baseline versus drift scenarios to assess the impact of data drift on model accuracy and reliability.

In [ ]:
# load trained model for prediction comparison
model, feature_columns = load_model_and_data()

# make predictions on baseline and drift data
baseline_features = baseline_data[feature_columns]
drift_features = drift_scenario_1[feature_columns]

baseline_predictions = model.predict(baseline_features)
drift_predictions = model.predict(drift_features)

print("prediction statistics:")
print(f"baseline predictions mean: {baseline_predictions.mean():.2f}")
print(f"drift scenario predictions mean: {drift_predictions.mean():.2f}")
print(f"prediction shift: {(drift_predictions.mean() - baseline_predictions.mean()):.2f}")

## Performance Degradation Analysis

Quantify model performance degradation using standard regression metrics to establish monitoring thresholds and trigger automated alerts for significant performance decline.

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# calculate model performance on baseline vs drift data
baseline_mse = mean_squared_error(baseline_data['MedHouseVal'], baseline_predictions)
baseline_mae = mean_absolute_error(baseline_data['MedHouseVal'], baseline_predictions)

drift_mse = mean_squared_error(drift_scenario_1['MedHouseVal'], drift_predictions)
drift_mae = mean_absolute_error(drift_scenario_1['MedHouseVal'], drift_predictions)

print("model performance comparison:")
print(f"baseline mse: {baseline_mse:.4f}")
print(f"drift scenario mse: {drift_mse:.4f}")
print(f"mse degradation: {((drift_mse - baseline_mse) / baseline_mse * 100):.2f}%")
print()
print(f"baseline mae: {baseline_mae:.4f}")
print(f"drift scenario mae: {drift_mae:.4f}")
print(f"mae degradation: {((drift_mae - baseline_mae) / baseline_mae * 100):.2f}%")

## Automated Monitoring Summary

Generate comprehensive monitoring summaries for dashboard integration and automated alerting systems. This summary provides key metrics and alert triggers for production monitoring workflows.

In [ ]:
# create monitoring summary
monitoring_summary = {
    'timestamp': datetime.now().isoformat(),
    'data_drift_detected': drift_results['metrics'][0]['result']['dataset_drift'],
    'drifted_features': drift_results['metrics'][0]['result']['number_of_drifted_columns'],
    'baseline_mse': baseline_mse,
    'current_mse': drift_mse,
    'performance_degradation_pct': ((drift_mse - baseline_mse) / baseline_mse * 100),
    'alert_triggered': drift_results['metrics'][0]['result']['dataset_drift'] or 
                     (((drift_mse - baseline_mse) / baseline_mse * 100) > 10)
}

print("monitoring summary:")
for key, value in monitoring_summary.items():
    print(f"{key}: {value}")

# save summary for dashboard
import json
with open('../reports/monitoring_summary.json', 'w') as f:
    json.dump(monitoring_summary, f, indent=2)

print("\nmonitoring summary saved for dashboard")